In [ ]:
!pip install -U tfx

     |████████████████████████████████| 2.5 MB 4.6 MB/s 
     |████████████████████████████████| 40 kB 3.8 MB/s 
     |████████████████████████████████| 10.2 MB 34.6 MB/s 
     |████████████████████████████████| 433 kB 61.2 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
     |████████████████████████████████| 1.4 MB 39.1 MB/s 
     |████████████████████████████████| 1.8 MB 41.7 MB/s 
     |████████████████████████████████| 133 kB 11.7 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 147 kB 45.2 MB/s 
     |████████████████████████████████| 19.2 MB 81.2 MB/s 
     |████████████████████████████████| 6.6 MB 24.4 MB/s 
     |████████████████████████████████| 206 kB 50.1 MB/s 
     |████████████████████████████████| 1.7 MB 40.9 MB/s 
     |████████████████████████████████| 1.8 MB 39.1 MB/s 
     |████████████████████████████████| 135 kB 66.1 MB/s 
     |████████████████████████████████| 23.6 MB 1.2 MB/s 
     |███████████

In [ ]:
import os
import tempfile
import urllib
import tensorflow_data_validation as tfdv
import pandas as pd

_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-data1t0ifyo8/data.csv', <http.client.HTTPMessage at 0x7f4c358db050>)

In [ ]:
# 머신러닝 파이프라인에 통합하기
from tfx import v1 as tfx
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2

# 학습, 평가, 테스트 데이터셋을 각각 6:2:2의 비율로 분할 / 분할 비율은 hash_buckets이 담당 / 기본 분할은 2:1 (학습, 평가)
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
                                                     example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
                                                     example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2),
    ])
)
context = InteractiveContext()

# 파이프라인 컴포넌트를 인스턴스화합니다.
example_gen = CsvExampleGen(input_base=_data_root, output_config=output)
# 대화식으로 컴포넌트를 실행합니다.
context.run(example_gen)

In [ ]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [ ]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=True)
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'payment_type',STRING,required,,'payment_type'
'dropoff_census_tract',INT,required,,-
'dropoff_community_area',INT,required,,-
'dropoff_latitude',FLOAT,required,,-
'dropoff_longitude',FLOAT,required,,-
'fare',FLOAT,required,,-
'pickup_census_tract',INT,required,,-
'pickup_community_area',INT,required,,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation', '3385 - 23210 Eman Cab', '3897 - 57856 Ilie Malec', '4623 - Jay Kim', '5006 - Salifu Bawa', '5129 - 98755 Mengisti Taxi', '585 - 88805 Valley Cab Co', '4053 - 40193 Adwar H. Nikola', '585 - Valley Cab Co', '5874 - Sergey Cab Corp.', '6742 - 83735 Tasha ride inc', '6743 - Luhak Corp'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown', 'Prcard'"


In [ ]:
# ExampleVlidator
from tfx.components import ExampleValidator

example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                     schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))